In [1]:
#define a function to calculate the locational margina cost for all busses in a network

#this part of the code is to upload the network with all the constrains

import pandapower as pp
import numpy as np
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net,name=1, vn_kv=220.)
bus2 = pp.create_bus(net,name=2, vn_kv=110.)
bus3 = pp.create_bus(net,name=3, vn_kv=110.)
bus4 = pp.create_bus(net,name=4, vn_kv=110.)

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV") #we havent

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_mw=60, controllable=False)
pp.create_load(net, bus3, p_mw=70, controllable=False)
pp.create_load(net, bus4, p_mw=10, controllable=False)

#create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=-1000, max_p_mw=1000)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80,  vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.01, controllable=True)

net.trafo.loc[net.trafo.index[0],"vkr_percent"]=0
net.trafo.loc[net.trafo.index[0],"pfe_kw"]=0

costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=10)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

net.poly_cost.cp1_eur_per_mw.at[costeg] = 10
net.poly_cost.cp1_eur_per_mw.at[costgen1] = 15
net.poly_cost.cp1_eur_per_mw.at[costgen2] = 12

net.trafo["max_loading_percent"] = 50

net.line["max_loading_percent"] = 50

net.bus["min_vm_pu"] = 1.0
net.bus["max_vm_pu"] = 1.02

pp.runopp(net, delta=1e-16)

In [2]:
def marginal_cost(net):   

    cost1=net.res_cost # we calculate the costs before increasing the power on every bus
    local=0
    for x in range(len(net.bus)):
        pp.create_load(net, bus=x, p_mw=1, controllable=False)
        pp.runopp(net, delta=1e-16)
        cost2=net.res_cost
        marginal=cost2-cost1
        print("The marginal cost for bus "+str(x+1)+" is: "+str(marginal)+" $/MWh")
        pp.create_load(net, bus=x, p_mw=-1, controllable=False) #this line deletes the previous augmented mega watt

In [3]:
marginal_cost(net)

#Results: for buses 1, 3 and 4 it is clear that the costs will be 10, 15 and 12, because those are the prices of production for the nearest power sources
#that means, the generation is taken from the closest source. For bus 2 though, it takes a different percentage of every source

The marginal cost for bus 1 is: 10.000004098038062 $/MWh
The marginal cost for bus 2 is: 13.139940911419217 $/MWh
The marginal cost for bus 3 is: 14.999595558321062 $/MWh
The marginal cost for bus 4 is: 11.999847066879056 $/MWh
